In [2]:
import trainer.glue_base as glue_base
import models.sparse_token as sparse
import pickle, importlib
importlib.reload(glue_base)
importlib.reload(sparse)
Glue = glue_base.GlueAttentionApproxTrainer

In [3]:
subsets = ["cola","mnli","mrpc","qnli","qqp","rte","sst2","stsb","wnli",]
subsets = ["mnli","mrpc","qnli","qqp","rte","sst2","stsb"]
subsets = ["mrpc"]
kss = [
    0.1, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.999,
]
ks = [0.1, 0.25, 0.375, 0.5,]
sparse.benchmark_reset()
# subsets = ["mrpc"]
# kss = ['dynamic:avg:avg:f',0.1]

def get_score(score):
    if 'accuracy' in score:
        return score['accuracy'], "acc"
    first_metric = list(score.keys())[0]
    return score[first_metric], first_metric

results = {}
i = 0
for subset in subsets:
    trainer = Glue(dataset=subset, factor=16, batch_size=-1, device=0)
    trainer.load()
    scores = {}
    metric_name = ""
    bert_score, metric_name = get_score(trainer.eval_base_model())
    scores['bert'] = f'{bert_score:.5f}'
    for ks in kss:
        sparse.benchmark_reset()
        sparse_score, _ = get_score(trainer.eval_sparse_model(ks=ks, use_forward=True))
        if isinstance(ks, str) and ks.startswith('dynamic'):
            est_k = sparse.benchmark_get_average('est_k')
            scores[str(ks)] = f'{sparse_score:.5f} (k:{est_k:.2f})'
        else:
            scores[str(ks)] = f'{sparse_score:.5f}'
        i += 1
        count = len(subsets) * len(kss)
        print(f'{i}/{count} | {subset} {ks} = {sparse_score:.5f}')
    results[f"{subset} ({metric_name})"] = scores

with open('glue_benchmark_forward.pkl', 'wb') as f:
    pickle.dump(results, f)

sparse.benchmark_report()

Trainer: mrpc


Downloading: 100%|██████████| 516/516 [00:00<00:00, 889kB/s]
Downloading: 100%|██████████| 418M/418M [00:17<00:00, 24.5MB/s] 
Reusing dataset glue (/home/ainl/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at /home/ainl/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-9433d17ba13b5e5f.arrow
Loading cached processed dataset at /home/ainl/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-a94cfdbaf3fde127.arrow
Reusing dataset glue (/home/ainl/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at /home/ainl/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-7814f5729032a0b8.arrow
Loading cached processed dataset 

Trainer: Save checkpoint path saves/glue-mrpc-16.pth


eval: 100%|██████████| 216/216 [00:03<00:00, 60.41it/s]


metric score {'accuracy': 0.8440579710144928, 'f1': 0.8865457612821593}
avg occupy 0.5399536083129833


...

metric score {'accuracy': 0.5797101449275363, 'f1': 0.5777518928363424}
avg occupy 0.5399536083129833
2/8 | mrpc 0.25 = 0.57971


eval: 100%|██████████| 216/216 [01:01<00:00,  3.53it/s]


metric score {'accuracy': 0.7878260869565218, 'f1': 0.8319559228650139}
avg occupy 0.5399536083129833
3/8 | mrpc 0.375 = 0.78783


eval:   5%|▌         | 11/216 [00:03<01:08,  3.00it/s]


KeyboardInterrupt: 

In [1]:
import pickle
import pandas as pd

with open('glue_benchmark_forward.pkl', 'rb') as f:
    results = pickle.load(f)

data = []
subsets = list(results.keys())
factors = list(results[subsets[0]].keys())
for factor in factors:             
    row = []
    for subset in subsets:
        row.append(results[subset][factor])
    data.append(row)

#remove cola and wnli
if subsets[0] == "cola (matthews_correlation)":
    for line in data:
        line.pop(0)
    subsets.pop(0)
    print("drop cola")

if subsets[-1] == "wnli (acc)":
    for line in data:
        line.pop(-1)
    subsets.pop(-1)
    print("drop wnli")

#calculate reproducibility
data_scalar = []
for line in data:
    xs = []
    for item in line:
        xs.append(float(item.split()[0]))
    data_scalar.append(xs)
reproducibilities = []
for i in range(len(data_scalar)):
    rsum = 0
    for k in range(len(data_scalar[i])):
        rsum += data_scalar[i][k]/data_scalar[0][k]
    rsum /= len(data_scalar[i])
    reproducibilities.append(rsum)
for i, r in enumerate(reproducibilities):
    data[i].append(f"{r*100:.2f}")
subsets.append("reproduce")

df = pd.DataFrame(data, columns=subsets, index=factors)
tex = df.to_latex()
with open('saves_plot/glue_benchmark_forward.tex', 'w') as f:
    f.write(tex)
df

drop cola
drop wnli


/tmp/ipykernel_123192/3086650456.py:48: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  tex = df.to_latex()


,mnli (acc),mrpc (acc),qnli (acc),qqp (acc),rte (acc),sst2 (acc),stsb (pearson),reproduce
bert,0.84208,0.84406,0.91543,0.90908,0.72563,0.92431,0.88047,100.00
0.1,0.49638,0.33681,0.62328,0.66876,0.54874,0.66284,0.20566,58.74
0.25,0.78686,0.41797,0.85210,0.88078,0.66426,0.83142,0.79653,86.41
0.375,0.83209,0.58841,0.90445,0.90566,0.68953,0.87385,0.86073,93.47
0.5,0.83984,0.71420,0.91323,0.90846,0.72563,0.90711,0.87449,97.36
0.625,0.84147,0.80464,0.91506,0.90903,0.72202,0.91858,0.87796,99.12
0.75,0.84187,0.83942,0.91525,0.90915,0.72563,0.92202,0.87986,99.87
0.875,0.84208,0.84406,0.91525,0.90913,0.72563,0.92431,0.88047,100.00
0.999,0.84208,0.84406,0.91543,0.90908,0.72563,0.92431,0.88047,100.00


In [ ]:
trainer = Glue(dataset='qnli', factor=16, batch_size=-1, device=0)
trainer.load() 

Trainer: qnli


Reusing dataset glue (C:\Users\AinL\.cache\huggingface\datasets\glue\qnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\qnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-95cdbfccba10767f.arrow
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\qnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-a121d057d8b0504c.arrow
Reusing dataset glue (C:\Users\AinL\.cache\huggingface\datasets\glue\qnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\qnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-81af1b1d0da19e67.arrow
Loading cached processed dataset at C:\Users\AinL\.cache\huggingface\datasets\glue\qnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9d

In [ ]:
trainer.model.bert = trainer.model_bert
trainer.eval_base_model()

eval: 100%|██████████| 683/683 [00:25<00:00, 26.83it/s]

metric score {'accuracy': 0.9154310818231741}
avg occupy 0.30243502260079336


{'accuracy': 0.9154310818231741}

In [ ]:
# trainer.seed()
# import models.sparse_token as sparse
# import transformers.models.bert.modeling_bert as berts
# import importlib
# importlib.reload(sparse)

# wrapped_bert = sparse.ApproxSparseBertModel(trainer.model_bert, approx_bert=trainer.approx_bert, ks=0.1)
# sparse_cls_bert = berts.BertForSequenceClassification(trainer.model_bert.config)
# sparse_cls_bert.load_state_dict(trainer.model.state_dict())
# sparse_cls_bert.bert = wrapped_bert
# sparse_cls_bert.to(trainer.device).eval()

# trainer.eval_base_model(model = sparse_cls_bert)

eval: 100%|██████████| 683/683 [00:36<00:00, 18.53it/s]


metric score {'accuracy': 0.5996705107084019}
avg occupy 0.30243502260079336


{'accuracy': 0.5996705107084019}